### 加载数据集

In [156]:
from utils import load_corpus, stopwords

TRAIN_PATH = "./data/weibo_dataset/train.txt"
TEST_PATH = "./data/weibo_dataset/test.txt"

In [157]:
# 分别加载训练集和测试集
train_data = load_corpus(TRAIN_PATH)
test_data = load_corpus(TEST_PATH)

In [158]:
import pandas as pd

df_train = pd.DataFrame(train_data, columns=["words", "label"])
df_test = pd.DataFrame(test_data, columns=["words", "label"])
df_train.head()

,words,label
0,书中 自有 黄金屋 书中 自有 颜如玉 沿着 岁月 的 长河 跋涉 或是 风光旖旎 或是 姹...,1
1,这是 英超 被 黑 的 最惨 的 一次 二哈 二哈 十几年来 中国 只有 孙继海 董方卓 郑...,0
2,中国 远洋 海运 集团 副总经理 俞曾 港 月 日 在 上 表示 中央 企业 走 出去 是 ...,1
3,看 流星花园 其实 也 还好 啦 现在 的 观念 以及 时尚 眼光 都 不一样 了 或许 十...,1
4,汉武帝 的 罪己 诏 的 真实性 尽管 存在 着 争议 然而 轮台 罪己 诏 作为 中国 历...,1


### 特征编码

In [159]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern='\[?\w+\]?', 
                             stop_words=stopwords,
                             max_features=2000)
X_train = vectorizer.fit_transform(df_train["words"])
y_train = df_train["label"]

d:\Python3.10.0\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['元', '吨', '数', '末'] not in stop_words.
  warnings.warn(


In [160]:
X_test = vectorizer.transform(df_test["words"])
y_test = df_test["label"]

### 训练模型&测试

In [161]:
import sys
!{sys.executable} -m pip install xgboost

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: d:\Python3.10.0\python.exe -m pip install --upgrade pip


In [162]:
import xgboost as xgb

param = {
    'booster':'gbtree',
    'max_depth': 17, 
    'scale_pos_weight': 0.5,
    'colsample_bytree': 0.8,
    'objective': 'binary:logistic',
    'eval_metric': 'error',
    'eta': 0.1,
    'nthread': 10,
}
dmatrix = xgb.DMatrix(X_train, label=y_train)
model = xgb.train(param, dmatrix, num_boost_round=200)

In [163]:
# 在测试集上用模型预测结果
dmatrix = xgb.DMatrix(X_test)
y_pred = model.predict(dmatrix)

In [164]:
# 测试集效果检验
from sklearn import metrics

auc_score = metrics.roc_auc_score(y_test, y_pred)          # 先计算AUC
y_pred = list(map(lambda x:1 if x > 0.5 else 0, y_pred))   # 二值化
print(metrics.classification_report(y_test, y_pred, digits=3))
print("准确率: {:.3f}".format(metrics.accuracy_score(y_test, y_pred)))
print("AUC:", auc_score)

              precision    recall  f1-score   support

           0      0.756     0.819     0.786       155
           1      0.916     0.881     0.898       345

    accuracy                          0.862       500
   macro avg      0.836     0.850     0.842       500
weighted avg      0.866     0.862     0.863       500

准确率: 0.862
AUC: 0.9096026180458158


### 手动输入句子，判断情感倾向（1正/0负）

In [165]:
from utils import processing

strs = ["哈哈哈哈哈笑死我了", "我也是有脾气的!"]
words = [processing(s) for s in strs]
vec = vectorizer.transform(words)
dmatrix = xgb.DMatrix(vec)

In [166]:
output = model.predict(dmatrix)
output

array([0.8925692 , 0.36707848], dtype=float32)